In [4]:
import os
os.chdir("../")

In [28]:
%pwd

'c:\\Users\\Excalibur\\chatbot-medic\\Medical-Chatbot'

In [27]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [30]:
#Extract data from pdf file

def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [17]:
%pwd

'c:\\Users\\Excalibur\\chatbot-medic\\Medical-Chatbot'

In [31]:
extracted_data = load_pdf(data = 'data/')

In [32]:
len(extracted_data)

599

In [33]:
#Split data into chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                 chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [34]:
text_chunks = text_split(extracted_data)
print("len of chunks", len(text_chunks))

len of chunks 6786


In [35]:
#Download the Embeddings from Hugging Face
from langchain.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [36]:
embeddings = download_hugging_face_embeddings()

C:\Users\Excalibur\AppData\Local\Temp\ipykernel_14960\2387794173.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Excalibur\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Excalibur\anaconda3\envs\medibot\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and 

In [70]:
from dotenv import load_dotenv
load_dotenv()

True

In [59]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [50]:
#create index
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="054c382e-cd93-44eb-af63-4e90c2c8c20e")

index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [64]:
import os
os.environ["PINECONE_API_KEY"] = "054c382e-cd93-44eb-af63-4e90c2c8c20e"

In [65]:
#Embed each chunk and send the embedding to Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [66]:
# Loading exisiting index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [71]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [72]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY


In [74]:
# Load the model
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [76]:
#initialize Retriever
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [75]:
# create RAG

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "you are a medibot man"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [77]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [80]:
response = rag_chain.invoke({"input": "What is universe?"})
print(response["answer"])


System: The universe is everything that exists, including all matter, energy, and space. It is constantly changing and evolving, and is believed to be interconnected and interdependent. In Daoism, the universe is seen as a manifestation of qi, or life force, and is in a state of constant flux.
